# FLOWCELL | OPERATION

[AAAA-MM-JJ] Suivi des opérations. Renseigner les noms de la _flowcell_ et de l'exérience (informations dans la SampleSheet, communiquées par courriel du CQGC ou récupérées de BaseSpace).

Cette ligne et les suivantes de cette cellule sont à titre informatives et peuvent être supprimées du journal.

0. Mise-en-place de l'environnement de travail
    1. Déconvoluer et convertir les BCLs en FASTQs `dragen_bcl-convert_watcher.sh`
1. Préparer les cas `run_pipeline_prag.sh`
    1. Collecter les informations sur les familles
    2. Téléverser les FASTQs sur BaseSpace
    3. Générer le fichier "emg_batch_manifest.csv"
2. Glisser-déposer le fichier "emg_batch_manifest.csv" dans Emedgene
    1. S'authentifier sur [Emedgene](https://chusaintejustine.emedgene.com/)
    2. Cliquer sur _+ New Case > Switch to batch_ (coin supérieur droit)
    3. Glisser-déposer le fichier "emg_batch_manifest.csv" dans la section _Select file_
3. Collecter les metriques
4. Archiver les résultats
5. Nettoyer

_C.f._ `README-PRAGMatIQ_Process.md` pour plus de détails et le dépannage.

In [ ]:
## 0. Mise en place de l'environnement de travail
# ssh ${USER}@10.128.80.26

screen -S prag
conda activate CQGC-utils
export FC=""
export XP=""
a=($(echo ${FC} | tr '_' '\n'))
export BASEDIR="/mnt/vs_nas_chusj/CQGC_PROD/sequenceurs/${a[1]}"
export WORKDIR="/mnt/spxp-app02/staging2/dragen"

## 1. Préparer les cas
## Normalement, le demux est automatique par "dragen_bcl-convert_watcher.sh" en cron
## Le pipeline téléverse les FASTQs sur BaseSpace et prépare "emg_batch_manifest.csv"
## à partir de "samples_list.csv" qu'il construit avec les informations dans Nanuq.

mkdir ${WORKDIR}/${FC} && cd $_
bash /staging2/soft/CQGC-utils/Analysis.pipeline_illumina/run_pipeline_prag.sh ${FC} 2>&1 | tee run_pipeline_prag.log
echo "delta"

## 2. Glisser-déposer le fichier "emg_batch_manifest.csv" dans [Emedgene](https://chusaintejustine.emedgene.com/)

In [ ]:
## 3. Collecter les metriques

python /staging2/soft/CQGC-utils/Analysis.pipeline_illumina/emg_collect_samples_metrics.py ${FC}

In [ ]:
## 4. Archiver les résultats

ssh hien@narval.calculquebec.ca

samples=()
total_samples=${#samples[@]}

cd ${HOME}/projects/ctb-rallard/COMMUN/PRAGMatIQ-EMG
cp archive_PRAGMatIQ.log archive_PRAGMatIQ.log0

count=1
for sample in ${samples[@]}; do
    echo "Processing sample ${count}/${total_samples}, ${sample}"
    aws s3 --profile emedgene cp s3://cac1-prodca-emg-auto-results/CHU_Sainte_Justine/${sample}/ ./archives/${sample} --recursive
    echo ${sample} $( date +'%Y-%m-%d %T' ) >> archive_PRAGMatIQ.log
    ((count++))
done

In [ ]:
# 6. Nettoyer
# Avant d'effacer le répertoire de la run, otifier le laboratoire qui doit sauvegarder quelqeues métriques

`rm -R ${BASEDIR}/${FC}`

## Logs